In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'D:\\Application\\spark-2.4.7-bin-hadoop2.7'

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('Linear_regression_sample').getOrCreate()
data = spark.read.option("sep", ",").csv('Ecommerce_Customers.csv', header=True, inferSchema=True)
data.show(3)

+--------------------+------------------+------+------------------+------------------+-----------------+--------------------+-------------------+
|               Email|           Address|Avatar|Avg Session Length|       Time on App|  Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+------------------+------+------------------+------------------+-----------------+--------------------+-------------------+
|mstephenson@ferna...|  835 Frank Tunnel|  null|              null|              null|             null|                null|               null|
|         Wrightmouth|    MI 82180-9605"|Violet| 34.49726772511229|12.655651149166752|39.57766801952616|   4.082620632952961|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Common|  null|              null|              null|             null|                null|               null|
+--------------------+------------------+------+------------------+------------------+-----------------+--------------------

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
for item in data.take(2)[1]:
    print(item)

Wrightmouth
 MI 82180-9605"
Violet
34.49726772511229
12.655651149166752
39.57766801952616
4.082620632952961
587.9510539684005


In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [8]:
assembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol = 'features')
output = assembler.transform(data)

In [9]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [11]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [12]:
train, test = final_data.randomSplit([0.7, 0.3])

In [14]:
lr = LinearRegression(labelCol = 'Yearly Amount Spent')
lr_model = lr.fit(train)

Py4JJavaError: An error occurred while calling o123.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<Avg Session Length:double,Time on App:double,Time on Website:double,Length of Membership:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spar

In [ ]:
test_result = lr_model.evaluate(test)

In [ ]:
test_result.rootMeanSquaredError

In [ ]:
test_result.r2

In [ ]:
unlabeled_data = test.select('features')
unlabeled_data.show()

In [ ]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()